# Introduction To K-Nearest Neighbors

## Objective

Use the kNN algorithm to identify the NBA player who's the most similar to Lebron James.

## Data Set

The data set contains information on how a player performed during the 2013-2014 NBA season.

Here are a few of the columns:

    •	player - The player's name
    •	pos - The player's position
    •	g - The number of games the player was in
    •	gs - The number of games in which the player started
    •	pts - The total points the player scored

See [this site](http://www.databasebasketball.com/about/aboutstats.htm) for descriptions of the remaining columns.


## Reading In The Data

In [1]:
import pandas as pd
nba_raw = pd.read_csv("C:/Users/i7/csv/nba_2013.csv")

# Replace NaN values with zeros.
nba = nba_raw.fillna(0)

# Convert strings to NaN and drop.
nba = nba.convert_objects(convert_numeric=True).dropna()
    
# The names of the columns in the data.
print("nba.columns.values:", nba.columns.values)

nba.head(5)

nba.columns.values: ['Rk' 'Player' 'Pos' 'Age' 'Tm' 'G' 'GS' 'MP' 'FG' 'FGA' 'FG%' '3P' '3PA'
 '3P.1' '2P' '2PA' '2P.1' 'eFG%' 'FT' 'FTA' 'FT%' 'ORB' 'DRB' 'TRB' 'AST'
 'STL' 'BLK' 'TOV' 'PF' 'PTS']


F:\ANACONDA\lib\site-packages\ipykernel\__main__.py:8: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1.0,Quincy Acy,PF,22.0,TOR,29.0,0.0,342.0,42.0,75.0,...,0.816,30.0,47.0,77.0,11.0,13.0,15.0,17.0,53.0,116.0
1,2.0,Jeff Adrien,PF,26.0,CHA,52.0,5.0,713.0,72.0,168.0,...,0.650,68.0,128.0,196.0,36.0,18.0,27.0,32.0,80.0,209.0
2,3.0,Arron Afflalo,SF,27.0,ORL,64.0,64.0,2307.0,397.0,905.0,...,0.857,29.0,210.0,239.0,206.0,40.0,11.0,138.0,137.0,1057.0
3,4.0,Josh Akognon,PG,26.0,DAL,3.0,0.0,9.0,2.0,4.0,...,0.000,0.0,1.0,1.0,1.0,0.0,0.0,0.0,3.0,5.0
4,5.0,Cole Aldrich,C,24.0,TOT,45.0,0.0,388.0,44.0,80.0,...,0.600,30.0,90.0,120.0,9.0,5.0,23.0,23.0,60.0,100.0


## Finding Similar Rows With Euclidean Distance

In [2]:
# Select Lebron James from the dataset
selected_player = nba[nba["Player"] == "LeBron James"]

In [3]:
selected_player

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
264,212.0,LeBron James,PF,28.0,MIA,76.0,76.0,2877.0,765.0,1354.0,...,0.753,97.0,513.0,610.0,551.0,129.0,67.0,226.0,110.0,2036.0


In [4]:
selected_player_2 = nba[nba["Player"] == "LeBron James"].iloc[0]

In [5]:
selected_player_2

Rk                 212
Player    LeBron James
Pos                 PF
Age                 28
Tm                 MIA
G                   76
GS                  76
MP                2877
FG                 765
FGA               1354
FG%              0.565
3P                 103
3PA                254
3P.1             0.406
2P                 662
2PA               1100
2P.1             0.602
eFG%             0.603
FT                 403
FTA                535
FT%              0.753
ORB                 97
DRB                513
TRB                610
AST                551
STL                129
BLK                 67
TOV                226
PF                 110
PTS               2036
Name: 264, dtype: object

In [6]:
# Choose only the numeric columns (Use these to compute euclidean distance)

distance_columns = ['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA',
 '3P.1', '2P', '2PA', '2P.1', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST',
 'STL', 'BLK', 'TOV', 'PF', 'PTS']

In [7]:
import math

def euclidean_distance(row):
    inner_value = 0
    for k in distance_columns:
        inner_value += (row[k] - selected_player[k]) ** 2
    return math.sqrt(inner_value)

# Find the distance from each player in the dataset to lebron.
lebron_distance = nba.apply(euclidean_distance, axis=1)

In [8]:
print("lebron_distance[:5]:\n", lebron_distance[:5])

lebron_distance[:5]:
 0    3878.055056
1    3485.609452
2    1561.897265
3    4237.902735
4    3845.353715
dtype: float64


## Normalizing Columns

Variables which are much larger in absolute terms have the potential to have a larger impact on distance. This can be bad, because a variable having larger values doesn't necessarily make it better at predicting what rows are similar.

A simple way to deal with this is to normalize all the columns to have a mean of 0, and a standard deviation of 1. This will ensure that no single column has a dominant impact on the euclidean distance calculations.

In [9]:
nba_numeric = nba[distance_columns]

nba_normalized = (nba_numeric - nba_numeric.mean()) / nba_numeric.std()

nba_numeric.head(5)

,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,3P.1,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,22.0,29.0,0.0,342.0,42.0,75.0,0.560,1.0,2.0,0.5,...,0.816,30.0,47.0,77.0,11.0,13.0,15.0,17.0,53.0,116.0
1,26.0,52.0,5.0,713.0,72.0,168.0,0.429,0.0,2.0,0.0,...,0.650,68.0,128.0,196.0,36.0,18.0,27.0,32.0,80.0,209.0
2,27.0,64.0,64.0,2307.0,397.0,905.0,0.439,72.0,240.0,0.3,...,0.857,29.0,210.0,239.0,206.0,40.0,11.0,138.0,137.0,1057.0
3,26.0,3.0,0.0,9.0,2.0,4.0,0.500,1.0,2.0,0.5,...,0.000,0.0,1.0,1.0,1.0,0.0,0.0,0.0,3.0,5.0
4,24.0,45.0,0.0,388.0,44.0,80.0,0.550,0.0,0.0,0.0,...,0.600,30.0,90.0,120.0,9.0,5.0,23.0,23.0,60.0,100.0


In [10]:
nba_normalized.head(5)

,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,3P.1,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,-1.072090,-0.779918,-0.788082,-0.877945,-0.786532,-0.864840,1.275708,-0.692923,-0.744648,1.442072,...,0.623629,-0.345612,-0.684541,-0.600830,-0.716027,-0.658966,-0.254608,-0.773548,-0.563340,-0.764147
1,-0.126638,0.107234,-0.612946,-0.455442,-0.601922,-0.597891,0.051613,-0.714468,-0.744648,-1.283917,...,-0.158543,0.284652,-0.096167,0.020484,-0.519700,-0.513235,0.117009,-0.526808,-0.167436,-0.550369
2,0.109725,0.570095,1.453660,1.359842,1.398023,1.517615,0.145056,0.836742,1.215335,0.351676,...,0.816816,-0.362198,0.499470,0.244992,0.815321,0.127978,-0.378481,1.216818,0.668362,1.398917
3,-0.126638,-1.782785,-0.788082,-1.257173,-1.032679,-1.068641,0.715054,-0.692923,-0.744648,1.442072,...,-3.221262,-0.843189,-1.018679,-0.997636,-0.794557,-1.037865,-0.719130,-1.053186,-1.296497,-1.019301
4,-0.599364,-0.162769,-0.788082,-0.825559,-0.774225,-0.850488,1.182265,-0.714468,-0.761119,-1.283917,...,-0.394137,-0.345612,-0.372194,-0.376322,-0.731733,-0.892134,-0.006864,-0.674852,-0.460699,-0.800926


## Finding The Nearest Neighbor

Use the distance.euclidean function from scipy.spatial, a much faster way to calculate euclidean distance.

In [11]:
from scipy.spatial import distance

# Fill in NA values in nba_normalized.
nba_normalized.fillna(0, inplace=True)

# Find the normalized vector for lebron james.
lebron_normalized = nba_normalized[nba["Player"] == "LeBron James"]

# Find the distance between lebron james and everyone else.
euclidean_distances = nba_normalized.apply(lambda row: distance.euclidean(row, lebron_normalized), axis=1)
distance_frame = pd.DataFrame(data={"dist": euclidean_distances, "idx": euclidean_distances.index})
distance_frame.sort("dist", inplace=True)

second_smallest = distance_frame.iloc[1]["idx"]

most_similar_to_lebron = nba.loc[int(second_smallest)]["Player"]
print("most_similar_to_lebron:", most_similar_to_lebron)

most_similar_to_lebron: Russell Westbrook


F:\ANACONDA\lib\site-packages\ipykernel\__main__.py:12: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


## Generating Training And Testing Sets

In [12]:
import random
from numpy.random import permutation

# Randomly shuffle the index of nba.
random_indices = permutation(nba.index)

# Set a cutoff for how many items we want in the test set (in this case 1/3 of the items).
test_cutoff = math.floor(len(nba)/3)

# Generate the test set by taking the first 1/3 of the randomly shuffled indices.
test = nba.loc[random_indices[1:test_cutoff]]

# Generate the train set with the rest of the data.
train = nba.loc[random_indices[test_cutoff:]]

## Predictions Using Sklearn

Sklearn performs the normalization and distance finding automatically, and lets us specify how many neighbors we want to look at.

In [13]:
from sklearn.neighbors import KNeighborsRegressor

# The columns that will be making predictions with.
x_columns = ['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA',
 '3P.1', '2P', '2PA', '2P.1', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST',
 'STL', 'BLK', 'TOV', 'PF']
# The column that we want to predict.
y_column = ['PTS']

# Create the knn model.
knn = KNeighborsRegressor(n_neighbors=5)

# Fit the model on the training data.
knn.fit(train[x_columns], train[y_column])

# Make predictions on the test set using the fit model.
predictions = knn.predict(test[x_columns])

print("predictions[:5]:\n", predictions[:5])

predictions[:5]:
 [[ 1379.6]
 [  803.8]
 [  708.2]
 [   65.6]
 [  106. ]]


## Computing Error

Compute the mean squared error between actual and predictions.

In [14]:
actual = test[y_column]

mse = (((predictions - actual) ** 2).sum()) / len(predictions)

print("actual[:20]:\n", actual[:20])
print("mse:", mse)

actual[:20]:
         PTS
248  1289.0
149   861.0
184   673.0
470    58.0
55     24.0
514   188.0
274  1397.0
73     13.0
232   216.0
466   608.0
371   729.0
211   208.0
303   506.0
389   125.0
428   330.0
353  1022.0
153  2280.0
119   392.0
225   467.0
444     2.0
mse: PTS    4780.910737
dtype: float64
